## generate with chatGPT

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re
import os
import pdfplumber
import json


In [29]:
system_prompt = """
Given a set of technical specifications, please generate a corresponding JSON representation in this format/Structure: {"heading": "Technical specifications heading", "values": [/* List of specific values */]}. Let the 'heading' serves as a label, and 'values' represent specific details under each category. If no heading is included in the text, make your own headings that makes sense for the technical specifications and that groups several of the values. Add each attribute as a specific values in the list. Use the following example and format as a guide:

{
  "specifications": {
    "Component Specifications": [
      {"Component Name": "[Value]"},
      {"Another Component": "[Value]"}
    ],
    "Environmental Specifications": [
     { "Operating temperature": "[Min-Max] [Unit]"},
      {"Storage temperature": "[Min-Max] [Unit]"},
      {"Operating humidity": "[Max] [Unit] non-condensing"},
      {"Storage humidity": "Less than [Max] [Unit]"}
    ],
    "Power Requirements": [
      {"Voltage": "[Range] [Frequency] [Unit]"},
      {"Power consumption": "[Max] [Unit]"}
    ],
    "Interfaces": [
      {"Interface Type": "[Count]"},
      {"Another Interface": "[Count]"}
    ],
    "Data Outputs": [
      {"Message format": "[Format]"},
      {"Message type": "[Type]"}
    ]
  }
}
""" 

In [26]:
def generate_technical_specs(text):
    import os
    from openai import OpenAI
    from openai import AzureOpenAI
    import pandas as pd


    deployment_name = "gpt-4-turbo-1106-preview"  # This will correspond to the custom name you chose for your deployment when you deployed a model.


    client = AzureOpenAI(
        azure_endpoint="https://da-openai-test.openai.azure.com/",
        api_key="5a90b85ba420469c9b36438e238d70fe",
        api_version="2023-05-15",
    )

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""Given this set of technical specifications {text}, please generate a corresponding JSON representation in this format/Structure: "heading": "Techniocal specifications heading", "values": [/* List of specific values */]."""
                }
            ],
        response_format={"type": "json_object"},
    )

    return response.choices[0].message.content.strip()



In [27]:
number = 1
products = pd.read_excel("data/all_products_dropped_discontinuid.xlsx")
print(products.iloc[number]["Data sheets"])
print(products.iloc[number]["Technical Specifications"])
print(generate_technical_specs(products.iloc[number]["Technical Specifications"]))

https://www.kongsberg.com/contentassets/d106c9aa05de423a8b02cf5fae6d6ad1/160920_transducer_12-16_60.pdf
Resonant frequency: 12 kHz
Maximum transducer depth: 20 m
Cable length: 20 m
Cable diameter: 12.5 mm
Weight: 84 / 78 kg with/without cable
Storage temperature: -20 to 70 °C
Circular beamwidth: 16 deg (narrow), 60 deg (wide)
Directivity: 130 (narrow), 10 (wide)
Impedance: Nominal 60 ohms
Transmitting response: 142 dB re 1µPa per V
Receiving sensitivity: -168.5 dB re 1V per µPa
Electroacoustic efficiency: 0.60
Maximum pulse power input: 100 W
Maximum continuous power input: 4 W
{
  "specifications": {
    "Acoustic Performance": [
      {"Resonant frequency": "12 kHz"},
      {"Circular beamwidth": "16 deg (narrow), 60 deg (wide)"}
    ],
    "Transducer Characteristics": [
      {"Maximum transducer depth": "20 m"},
      {"Directivity": "130 (narrow), 10 (wide)"},
      {"Impedance": "Nominal 60 ohms"},
      {"Transmitting response": "142 dB re 1µPa per V"},
      {"Receiving sensit

In [34]:
# products = pd.read_excel("data/all_products_dropped_discontinuid.xlsx")
# products["formated_technical_specs"]=""
# products["formated_technical_specs"] = products["formated_technical_specs"].astype(str)
# products.to_excel("data/all_products_technical_formated.xlsx")


In [ ]:
import pandas as pd

# Load the products data
products = pd.read_excel("data/all_products_technical_formated.xlsx")

# Iterate over each row in the DataFrame
for index, row in products.iterrows():
        if pd.isna(row["formated_technical_specs"]):
            if not pd.isna(row["Technical Specifications"]):

                print(row["Product_Name"], "index:", index)
                print(row["Data sheets"])
                print(row["Technical Specifications"])
                json_output = (generate_technical_specs(row["Technical Specifications"]))
                json_output = json_output.replace('\n', '\n').replace('\r', '\n').replace('\t', '\n')

                try:
                    data = json.loads(json_output)
                    print(data)
                    products.loc[index, 'formated_technical_specs'] = json.dumps(data)
                except json.JSONDecodeError:
                    print(f'Error decoding JSON for index {index}: {json_output}')

                products.to_excel("data/all_products_technical_formated.xlsx")
                products.to_csv("data/all_products_technical_formated.csv")

## Format technical specs again

In [46]:
import pandas as pd

# Load the products data
products = pd.read_excel("data/12_15/All_Products_related.xlsx")

In [30]:
# Iterate over each row in the DataFrame
# check how many technical spec that are not formated
count = 0

       
        
            

[{'Note': 'Specifications subject to change without any further notice.'}]
Technical Specifications
[]
General Specifications
[{'Unknown Specification': 'Unknown Value'}]
Unknown Technical Specifications
['Error: Insufficient data to generate technical specifications. Please provide specific details or attributes of the technical specifications to be represented in JSON format.']
nan
No technical specifications provided.
Technical Specifications
[{'Unknown Specification': 'Please provide specific technical details to generate a JSON representation.'}]
nan
No technical specifications provided to generate JSON representation.
Incomplete technical specifications provided. Please provide detailed specifications to generate the corresponding JSON representation.
Specifications subject to change without any further notice.
nan
nan
['Laptop', 'Hydrographic Work Station (HWS)', 'Monitor']
General Technical Specifications
[{'Specification Detail': 'Unknown'}]
Specifications subject to change wi

In [47]:
import json

class InvalidInputError(Exception):
    pass

error_count = 0

def format_specs(input_data):
    if pd.isna(input_data):
        return []
    global error_count
    # Parse the input data into a Python object
    try: 
        input_data = json.loads(input_data)
    except json.JSONDecodeError:
        print(f"InvalidInputError: Could not parse input data as JSON: {input_data}")
        error_count += 1
        return []

    # Check if input_data is not a dictionary and raise an error if it is not
    if not isinstance(input_data, dict):
        print(f"InvalidInputError: Input data is not a dictionary: {input_data}")
        error_count += 1
        return []

    output_data = []
    error_occurred = False

    # Check for both 'Specifications' and 'specifications' keys
    specs_key = None
    if 'Specifications' in input_data:
        specs_key = 'Specifications'
    elif 'specifications' in input_data:
        specs_key = 'specifications'

    if specs_key is not None:
        for category, specs in input_data[specs_key].items():
            formatted_specs = []
            for spec in specs:
                # Check if spec is a dictionary before trying to call items() on it
                if isinstance(spec, dict):
                    title, value = list(spec.items())[0]
                    
                    # If value is a list, convert each item to a string and then join them without brackets
                    if isinstance(value, list):
                        value = ', '.join(str(item) for item in value)
                    
                    formatted_spec = {"Title": title, "Value": value}
                    formatted_specs.append(formatted_spec)
                else:
                    print(f"InvalidInputError: Expected a dictionary, but got {type(spec)}: {spec}")
                    error_count += 1
                    error_occurred = True
                    return []

            output_category = {"Title": category, "Specifications": formatted_specs}
            output_data.append(output_category)

    # Print an error message if output_data is empty
    if not output_data:
        print(f"InvalidInputError: No valid specifications found in input data: {input_data}")
        error_count += 1
        error_occurred = True

    if error_occurred:
        return []
    else:
        return output_data

products['formatted_specs'] = products['formated_technical_specs'].apply(format_specs)

print(f"Total errors: {error_count}")
products.to_excel("data/12_15/All_Products_related_technical.xlsx")

InvalidInputError: No valid specifications found in input data: {'heading': 'Technical Specifications', 'values': []}
InvalidInputError: No valid specifications found in input data: {'heading': 'General Specifications', 'values': [{'Unknown Specification': 'Unknown Value'}]}
InvalidInputError: No valid specifications found in input data: {'heading': 'Unknown Technical Specifications', 'values': ['Error: Insufficient data to generate technical specifications. Please provide specific details or attributes of the technical specifications to be represented in JSON format.']}
InvalidInputError: Expected a dictionary, but got <class 'str'>: 3
InvalidInputError: No valid specifications found in input data: {'error': 'No technical specifications provided.'}
InvalidInputError: No valid specifications found in input data: {'heading': 'Technical Specifications', 'values': [{'Unknown Specification': 'Please provide specific technical details to generate a JSON representation.'}]}
InvalidInputError

In [48]:
# print all the rows with empty [] in the formatted_specs column
new_prod = products[products['formatted_specs'].apply(lambda x: len(x) == 0)]
len(new_prod)

36

In [49]:
number = 2
input_data = products.iloc[number]["formated_technical_specs"]
output_data = format_specs(input_data)

In [53]:
products

,Unnamed: 0.1,Unnamed: 0,Product_Name,Product category,Introduction,Block1,Block2,Block3,formated_technical_specs,Features,...,downloads,product_family_name,is_range,grouped_data_sheets,Hero Image,Other Image,is_software,product_tags,related,formatted_specs
0,0,0,"Seapath 380 - utilising GPS, GLONASS, Galileo...","Seafloor mapping, Inertial solutions","{'heading': 'Introduction', 'text': 'The Seapa...","{'heading': 'Advanced Navigation Integration',...",{'heading': 'Functional Design and System Conf...,"{'heading': 'Product Range and Applications', ...","{""specifications"": {""Performance Power Specifi...",0.007° to 0.02° roll and pitch accuracy depend...,...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",NaN,NaN,NaN,NaN,NaN,False,"{""Seafloor mapping"": {""Area"": ""Underwater"", ""P...",NaN,"[{'Title': 'Performance Power Specifications',..."
1,1,1,12 kHz dual-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 12-16...","{'heading': 'Dual-Beam Functionality', 'text':...","{'heading': 'Beam Characteristics', 'text': 'T...",{'heading': 'Technical and Acoustic Performanc...,"{""specifications"": {""Transducer Characteristic...",Resonant frequency: 12 kHz;\nDual beam transdu...,...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/d106...,['https://www.kongsberg.com/contentassets/e811...,NaN,False,"{""Geophysical survey"": {""Area"": ""Underwater"", ...","{""Simrad 38-7"": ""https://www.kongsberg.com/mar...","[{'Title': 'Transducer Characteristics', 'Spec..."
2,2,2,120 kHz single-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 120-2...",{'heading': 'Order Number and Technical Overvi...,{'heading': 'Performance and Construction Deta...,{'heading': 'Acoustic Qualities and Efficiency...,"{""specifications"": {""Acoustic Properties"": [{""...","Resilient Design: Housed in a robust casing, t...",...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/d106...,['https://www.kongsberg.com/contentassets/e811...,NaN,False,"{""Seafloor mapping"": {""Area"": ""Underwater"", ""P...","{""Simrad 38-7"": ""https://www.kongsberg.com/mar...","[{'Title': 'Acoustic Properties', 'Specificati..."
3,3,3,18 kHz single-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 18-11...","{'heading': 'Key Specifications', 'text': 'The...",{'heading': 'Physical Attributes and Operating...,"{'heading': 'Manufacturing Details', 'text': '...","{""specifications"": {""Frequency and Response"": ...",Key Features:;\nResonant frequency: 18 kHz;\nT...,...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/d106...,['https://www.kongsberg.com/contentassets/e811...,NaN,False,"{""Geophysical survey"": {""Area"": ""Underwater"", ...","{""Simrad 38-7"": ""https://www.kongsberg.com/mar...","[{'Title': 'Frequency and Response', 'Specific..."
4,4,4,200 kHz single-beam transducer in robust housing,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 200-2...",{'heading': 'Robust Design & Deepwater Capabil...,{'heading': 'High Performance and Efficient Op...,{'heading': 'Durable Construction and Environm...,"{""specifications"": {""Acoustic characteristics""...",200 kHz single-beam transducer;\nRobust housin...,...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/d106...,['https://www.kongsberg.com/contentassets/e811...,NaN,False,"{""Geophysical survey"": {""Area"": ""Underwater"", ...","{""Simrad 38-7"": ""https://www.kongsberg.com/mar...","[{'Title': 'Acoustic characteristics', 'Specif..."
...,...,...,...,...,...,...,...,...,...,...,..

In [51]:
output_data

[{'Title': 'Acoustic Properties',
  'Specifications': [{'Title': 'Resonant frequency', 'Value': '120 kHz'},
   {'Title': 'Beamwidth, circular', 'Value': '10 deg. ±2 deg.'},
   {'Title': 'Directivity index', 'Value': '25 dB'},
   {'Title': 'Equivalent two-way beam angle (ψ)', 'Value': '-17.5 dB'},
   {'Title': 'Transmitting response', 'Value': '176.5 ±2 dB re 1µPa per V'},
   {'Title': 'Receiving sensitivity (open circuit)',
    'Value': '-183.0 ±2 dB re 1V per µPa'},
   {'Title': 'Electroacoustic efficiency', 'Value': '70 ±20 %'}]},
 {'Title': 'Power Specifications',
  'Specifications': [{'Title': 'Maximum input power', 'Value': '1000 W'},
   {'Title': 'Maximum duty cycle (at maximum power)', 'Value': '5%'}]},
 {'Title': 'Physical Specifications',
  'Specifications': [{'Title': 'Maximum installation depth',
    'Value': '1000 m'},
   {'Title': 'Cable length', 'Value': '25 m'},
   {'Title': 'Nominal impedance', 'Value': '60 ohm'},
   {'Title': 'Cable diameter', 'Value': '10.9 mm'},
   {

In [52]:
products_tech = pd.read_excel("data/all_products_technical_formated.xlsx")
count = 0
for index, row in products_tech.iterrows():
    if row["Technical Specifications"] == "I don't know":
        # Increment the count
        count += 1
        # Change the technical spec to empty string
        # products.loc[index, 'formated_technical_specs'] = str({"heading":"", "values":[]})
        # Change the product on the same row
        # Print the new values
        print(products.iloc[index]["formated_technical_specs"])
        print(products.iloc[index]["Product_Name"],"===",products_tech.iloc[index]["Product_Name"])


print(f'Count of "I don\'t know": {count}')

{"heading": "Technical Specifications", "values": []}
AIS service management system === AIS service management system
{"heading": "General Specifications", "values": [{"Unknown Specification": "Unknown Value"}]}
Analytics === Analytics
{"heading": "Unknown Technical Specifications", "values": ["Error: Insufficient data to generate technical specifications. Please provide specific details or attributes of the technical specifications to be represented in JSON format."]}
Aquaculture monitoring system === Aquaculture monitoring system
{"error": "No technical specifications provided."}
Blue Insight === Blue Insight
{"heading": "Technical Specifications", "values": [{"Unknown Specification": "Please provide specific technical details to generate a JSON representation."}]}
Core === Core
{"specifications": {"Unknown Category": [{"Unspecified Specification": "Unknown value"}]}}
cPAP Portable tranceivers === cPAP Portable tranceivers
{"error": "No technical specifications provided to generate J